In [1]:
data_var = '2024-02-17'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7285,2024-02-17,Brasil Nbb,11:00,São Paulo,Corinthians Paulista,1.41,2.76,160.5,1.85,1.85,-7.5,2.08,1.63,tdTlyKCL,0.709220,0.362319,0.540541,0.540541,0.071539,0.6,0.4,NaN,NaN,215.084,84.617210,0.393415,125.774,46.166284,0.367057,153.094,150.626,188.10,116.27,0.0,0.0,0.0,0.0,0.457839,0.000000,0.171535,-1.21,-0.242,-1.694215,0.620013,0.4,-0.220013,-0.51,-0.102,-17.254902,0.000000,0.0,0.000000
7286,2024-02-17,Brasil Nbb,17:10,Brasília,Flamengo,8.45,1.05,161.5,1.81,1.85,15.5,1.94,1.77,M7pnzvSR,0.118343,0.952381,0.552486,0.540541,0.070724,0.6,0.8,NaN,NaN,334.820,153.173759,0.457481,103.426,8.891320,0.085968,251.880,106.546,465.80,98.04,1.0,0.0,0.0,0.0,1.101598,0.015456,0.064802,-2.81,-0.562,-13.256228,0.000000,0.0,0.000000,-1.69,-0.338,-0.147929,0.804422,0.8,-0.004422
7287,2024-02-17,Brasil Nbb,18:00,Bauru,R10 Score Vasco da Gama,1.50,2.47,153.5,1.81,1.86,-5.5,2.04,1.69,OGWDVczq,0.666667,0.404858,0.552486,0.537634,0.071525,0.8,0.4,NaN,NaN,137.342,13.052834,0.095039,123.748,27.274188,0.220401,118.816,182.904,116.59,127.68,0.0,0.0,0.0,0.0,0.345538,0.019267,0.132701,1.65,0.330,1.515152,0.745016,1.0,0.254984,-1.35,-0.270,-5.444444,0.000000,0.0,0.000000
7288,2024-02-17,Brasil Nbb,18:00,Franca,Pato,1.05,8.25,164.5,1.80,1.90,-15.5,2.08,1.63,IwWHUHkk,0.952381,0.121212,0.555556,0.526316,0.073593,0.4,0.8,NaN,NaN,115.866,30.216924,0.260792,160.890,54.340613,0.337750,100.608,254.724,90.72,256.75,0.0,0.0,0.0,0.0,1.094875,0.038222,0.171535,0.45,0.090,0.555556,0.000000,0.0,0.000000,-1.35,-0.270,-26.851852,0.352600,0.3,-0.052600
7289,2024-02-17,Alemanha Pro A,14:00,Bremerhaven,Nurnberger,1.39,2.84,162.5,1.83,1.87,-7.5,2.04,1.69,f5YjNW2s,0.719424,0.352113,0.546448,0.534759,0.071537,0.8,0.8,NaN,NaN,162.578,40.635798,0.249946,178.960,40.527770,0.226463,193.194,210.052,100.80,218.08,0.0,0.0,0.0,0.0,0.484778,0.015289,0.132701,2.32,0.464,0.840517,0.000000,0.0,0.000000,-0.61,-0.122,-15.081967,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7543,2024-02-17,Itália Série A1 Feminina,16:30,Brixia F,Sanga Milano F,1.23,3.86,137.5,1.85,1.85,-10.5,2.03,1.70,z5LBrRx9,0.813008,0.259067,0.540541,0.540541,0.072075,0.0,0.4,NaN,NaN,369.652,330.181560,0.893223,420.474,188.489611,0.448279,0.000,565.412,205.36,371.78,0.0,0.0,0.0,0.0,0.730723,0.000000,0.125118,0.00,0.000,inf,0.000000,0.0,0.000000,-3.31,-0.662,-4.320242,0.000000,0.0,0.000000
7544,2024-02-17,Macedônia Do Norte Copa Da Macedônia,14:00,MZT Skopje,Pelister,1.91,1.81,154.5,1.81,1.89,-6.5,2.01,1.68,t4FeaDSQ,0.523560,0.552486,0.552486,0.529101,0.076046,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,130.41,0.00,0.0,0.0,0.0,0.0,0.038016,0.030578,0.126474,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7545,2024-02-17,Polônia Copa Da Polônia,15:00,Anwil Wloclawek,Ostrow Wielkopolski,1.45,2.67,163.5,1.86,1.88,-6.5,2.03,1.68,zHVd969H,0.689655,0.374532,0.537634,0.531915,0.064187,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,105.30,0.00,0.0,0.0,0.0,0.0,0.418772,0.007563,0.133416,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7546,2024-02-17,Sérvia Copa Korac,17:00,Crvena zvezda,Partizan,2.12,1.67,160.5,1.83,1.87,1.5,1.95,1.74,6cBkFYy8,0.471698,0.598802,0.546448,0.534759,0.070501,0.0,0.0,NaN,NaN,NaN,NaN

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
